In [1]:
# Must be included at the beginning of each new notebook. Remember to change the app name.
import findspark
findspark.init('/home/ubuntu/spark-2.1.1-bin-hadoop2.7')
import pyspark
from pyspark.sql import *
spark = SparkSession.builder.appName('Logistic').getOrCreate()

# If you're getting an error with numpy, please type 'sudo pip install numpy --user' into the EC2 console.
from pyspark.ml.classification import LogisticRegression
bank = spark.read.csv('/home/ubuntu/Loan Defaulter.csv', header=True, inferSchema=True)

In [2]:
# Print data schema.
bank.printSchema()

# Print data columns.
bank.columns

root
 |-- SK_ID_CURR: integer (nullable = true)
 |-- TARGET: integer (nullable = true)
 |-- CODE_GENDER: string (nullable = true)
 |-- CNT_CHILDREN: integer (nullable = true)
 |-- AMT_INCOME_TOTAL: double (nullable = true)
 |-- AMT_CREDIT: double (nullable = true)
 |-- AMT_ANNUITY: double (nullable = true)
 |-- AMT_GOODS_PRICE: double (nullable = true)
 |-- REGION_POPULATION_RELATIVE: double (nullable = true)
 |-- DAYS_BIRTH: integer (nullable = true)
 |-- DAYS_EMPLOYED: integer (nullable = true)
 |-- DAYS_REGISTRATION: integer (nullable = true)
 |-- DAYS_ID_PUBLISH: integer (nullable = true)
 |-- OWN_CAR_AGE: integer (nullable = true)
 |-- FLAG_MOBIL: integer (nullable = true)
 |-- FLAG_EMP_PHONE: integer (nullable = true)
 |-- FLAG_WORK_PHONE: integer (nullable = true)
 |-- FLAG_CONT_MOBILE: integer (nullable = true)
 |-- FLAG_PHONE: integer (nullable = true)
 |-- FLAG_EMAIL: integer (nullable = true)
 |-- CNT_FAM_MEMBERS: integer (nullable = true)
 |-- REGION_RATING_CLIENT: integer 

['SK_ID_CURR',
 'TARGET',
 'CODE_GENDER',
 'CNT_CHILDREN',
 'AMT_INCOME_TOTAL',
 'AMT_CREDIT',
 'AMT_ANNUITY',
 'AMT_GOODS_PRICE',
 'REGION_POPULATION_RELATIVE',
 'DAYS_BIRTH',
 'DAYS_EMPLOYED',
 'DAYS_REGISTRATION',
 'DAYS_ID_PUBLISH',
 'OWN_CAR_AGE',
 'FLAG_MOBIL',
 'FLAG_EMP_PHONE',
 'FLAG_WORK_PHONE',
 'FLAG_CONT_MOBILE',
 'FLAG_PHONE',
 'FLAG_EMAIL',
 'CNT_FAM_MEMBERS',
 'REGION_RATING_CLIENT',
 'REGION_RATING_CLIENT_W_CITY',
 'HOUR_APPR_PROCESS_START',
 'REG_REGION_NOT_LIVE_REGION',
 'REG_REGION_NOT_WORK_REGION',
 'LIVE_REGION_NOT_WORK_REGION',
 'REG_CITY_NOT_LIVE_CITY',
 'REG_CITY_NOT_WORK_CITY',
 'LIVE_CITY_NOT_WORK_CITY']

In [3]:
my_cols=bank.select(['SK_ID_CURR', 'TARGET',
 'CODE_GENDER',
 'CNT_CHILDREN','DAYS_EMPLOYED'])

In [4]:
my_final_data = my_cols.na.drop()

In [5]:
from pyspark.ml.feature import (VectorAssembler,VectorIndexer,
                                OneHotEncoder,StringIndexer)

In [6]:
gender_indexer = StringIndexer(inputCol='CODE_GENDER',outputCol='CODE_GENDERIndex')
gender_encoder = OneHotEncoder(inputCol='CODE_GENDERIndex',outputCol='CODE_GENDERVec')

In [7]:
assembler = VectorAssembler(inputCols=['SK_ID_CURR','TARGET','CODE_GENDERVec','CNT_CHILDREN','DAYS_EMPLOYED'],outputCol='features')

In [8]:
from pyspark.ml.classification import LogisticRegression

In [9]:
from pyspark.ml import Pipeline

In [10]:
log_reg_titanic = LogisticRegression(featuresCol='features',labelCol='TARGET')

In [11]:
pipeline = Pipeline(stages=[gender_indexer,gender_encoder, assembler, log_reg_titanic])

In [12]:
train_titanic_data, test_titanic_data = my_final_data.randomSplit([0.7,0.3])

In [13]:
fit_model = pipeline.fit(train_titanic_data)

IllegalArgumentException: 'requirement failed: Column CODE_GENDERVec must be of type NumericType but was actually of type org.apache.spark.ml.linalg.VectorUDT@3bfc3ba7.'

In [ ]:
results = fit_model.transform(test_titanic_data)

In [ ]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

my_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction',
                                       labelCol='CODE_GENDERVec')

In [ ]:
results.select('CODE_GENDERVec','prediction').show()

In [ ]:
AUC = my_eval.evaluate(results)

AUC